In [5]:
import numpy as np
from matplotlib import pyplot as plt
import time
import pylab as pl
from IPython import display
from parameters import *
from micro import *
from chem import *
from aero_init import *
from parcel import *
from condensation import *
from collision import *
from analysis import *
import itertools

import matplotlib

%load_ext autoreload
%autoreload 2

In [16]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
from IPython.display import clear_output

def spec_plot(ax, spectra_arr, nt, log_edges):
    # Clear the previous plot
    clear_output(wait=True)

    # Set the y-scale to logarithmic
    ax.set_yscale("log")

    # Create the contour plot with a logarithmic color scale
    contour = ax.contourf(np.arange(nt+1), log_edges[1:], spectra_arr.T, norm=matplotlib.colors.LogNorm())

    # Create a logarithmic colorbar
    cbar = plt.colorbar(contour, ax=ax, format="%.0e")

    # Add labels and title
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_title("Contour Plot with Log Scale")
    
    plt.show()
    
    
def print_output(t, z_parcel, T_parcel, q_parcel, qa, qc, qr, na, nc, nr):
    # Clear previous output
    clear_output(wait=True)

    # Print the initial variable names
    print("value: {:<8}  {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}".format(
        "Time (s)", "z (m)", "T (K)", "qv (g/kg)", "QA", "QC (g/kg)", "QR (g/kg)", "NA", "NC", "NR"))

    # Print the updated output
    print("after: {:<8.1f}  {:<8.2f} {:<8.2f} {:<9.2f} {:<8.2f} {:<8.2f} {:<8.2f} {:<8.2f} {:<8.2f} {:<8.2f}".format(
        t * dt + 1, z_parcel, T_parcel, 1e3 * q_parcel, 1e3 * qa, 1e3 * qc, 1e3 * qr, na / 1e6, nc / 1e6, nr / 1e6))

# Intialization
## Input parameter

In [17]:
n_ptcl = 1000


dt = 0.5
nt = 7200

do_condensation = True
do_collision    = False

#parcel info. 
T_parcel   = 293.2 
P_parcel   = 1013.0E2
q_parcel   = 8.0E-3
RH_parcel  = 0.88
w_parcel   = 0.5 # m/s
z_parcel   = 0.0 #m

max_z = 3600. 

#aerosol initialization
N_aero = [100.0E6, 50.0E6]
mu_aero = [np.log(100.0E-9), np.log(500.0E-9)]
sigma_aero = [np.log(2), np.log(2)]
rho_aero = 2170.0
molecular_weight_aero = 0.058443

n_particles_mode1 = n_ptcl // 2  # Number of particles for the first mode
n_particles_mode2 = n_ptcl - n_particles_mode1  # Number of particles for the second mode

# Generate log-normal distribution for the first mode
mode1_values = np.random.lognormal(mu_aero[0], sigma_aero[0], n_particles_mode1)

# Generate log-normal distribution for the second mode
mode2_values = np.random.lognormal(mu_aero[1], sigma_aero[1], n_particles_mode2)

# Combine the values from both modes
aero_r_seed = np.concatenate((mode1_values, mode2_values))

nbins = 100
log_edges = np.logspace(np.log10(0.1E-6), np.log10(200.0E-6), nbins + 1)
spectra_arr = np.zeros((nt+1,nbins))

# Time step routine

In [18]:
dz=0
z_parcel, T_parcel, rho_parcel, V_parcel, air_mass_parcel = ascend_parcel(z_parcel, T_parcel,P_parcel, dz)
#Aerosol init
T_parcel, q_parcel, particles_list = aero_init( n_ptcl, P_parcel, T_parcel,q_parcel, aero_r_seed,N_aero,rho_aero,molecular_weight_aero)
#parcel routine
spectra_arr[0],qa, qc,qr, na, nc, nr = qc_qr_analysis(particles_list,air_mass_parcel,log_edges)

fig, ax = plt.subplots()

for t in range(nt):
    
    #Parcel ascending
    dz = w_parcel * dt
    if z_parcel < max_z: 
        z_parcel, T_parcel, rho_parcel, V_parcel, air_mass_parcel = ascend_parcel(z_parcel, T_parcel,P_parcel, dz)
    
    #Condensational Growth
    dq_liq = 0.0
    if do_condensation:
        particles_list, T_parcel, q_parcel = drop_condensation(particles_list, T_parcel, q_parcel, P_parcel, dt, air_mass_parcel, rho_aero, molecular_weight_aero)

    #Collisional Growth
    if do_collision:
        particles_list = collection(dt, particles_list,rho_parcel, rho_liq, P_parcel, T_parcel)
    
    #Analysis
    spectra_arr[t+1],qa, qc,qr, na, nc, nr = qc_qr_analysis(particles_list,air_mass_parcel,log_edges)
    
    #Visulaization
    print_output(t, z_parcel, T_parcel, q_parcel, qa, qc, qr, na, nc, nr)
    spec_plot(ax,spectra_arr, nt, log_edges)
    # Show the plot
plt.show()

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [1]:
import ipywidgets as widgets
from IPython.display import display

# Create the widgets for the variables
var1_widget = widgets.Text(description='Variable 1:')
var2_widget = widgets.IntSlider(description='Variable 2:', min=0, max=100, value=50)
var3_widget = widgets.Checkbox(description='Variable 3:', value=False)

# Define a function to be called when the user clicks the button
def set_variables(button):
    var1 = var1_widget.value
    var2 = var2_widget.value
    var3 = var3_widget.value
    
    # Use the variables to initialize your model
    # ...
    
    # Print the values for demonstration purposes
    print(f'Variable 1: {var1}')
    print(f'Variable 2: {var2}')
    print(f'Variable 3: {var3}')

# Create a button widget
button = widgets.Button(description='Set Variables')

# Attach the button click event to the function
button.on_click(set_variables)

# Display the widgets and button
display(var1_widget, var2_widget, var3_widget, button)


Text(value='', description='Variable 1:')

IntSlider(value=50, description='Variable 2:')

Checkbox(value=False, description='Variable 3:')

Button(description='Set Variables', style=ButtonStyle())